In [ ]:
import h5py
from collections import deque
from pprint import pprint
from doct import Document
from metadataclient.api import (find_last, find_descriptors,
                                find_events, find_run_stops)
import ujson


In [ ]:
def get_descriptors():
    header = {}
    rstart = next(find_last())
    descs = []
    for d in find_descriptors(run_start=rstart['uid']):
        descs.append(d)
    return descs
def extract_data_keys(descriptor):
    dkeys = []
    for k, v in descriptor['data_keys'].items():
        dkeys.append(k)
    return dkeys

In [ ]:
descriptors = get_descriptors()

In [ ]:
for d in descriptors:
    f = h5py.File(d['uid'] + '.hdf5','w') # create file with given descriptor uid
    evs = deque()
    dkeys = extract_data_keys(d)
    vectorized = {}
    tmp = []
    for ev in find_events(descriptor=d):
        for k in dkeys:
            tmp.append(ev['data'][k])
    vectorized[k] = tmp
    print(vectorized)
    m_shape = (len(vectorized[k]),)# d['data_keys'][k]['shape'])
    print(m_shape)
    dset = f.create_dataset(k, shape=m_shape, dtype='float' )
    dset[:] = vectorized[k]